In [1]:
import itertools
import pickle
from collections import Counter

import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import plotly.graph_objects as go

In /home/pwielopolski/anaconda3/envs/datascience/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/pwielopolski/anaconda3/envs/datascience/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/pwielopolski/anaconda3/envs/datascience/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In /home/pwielopolski/anaconda3/envs/datascience/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be re

In [2]:
def flatten(data):
    return [x for el in data for x in el]

### Load data

In [3]:
with open('authors.pickle', 'rb') as f:
    data = pickle.load(f)

In [4]:
data

[['F.Chaari', 'J.Leskow', 'R.Zimroz', 'A.Wylomanska', 'A.Dudek'],
 ['F.Chaari', 'J.Leskow', 'A.Napolitano', 'R.Zimroz', 'A.Wylomanska'],
 ['R.G.Burns', 'I.Szymaniec', 'A.Weron'],
 ['R.G.Burns', 'I.Szymaniec', 'A.Weron'],
 ['F.Chaari',
  'J.Leskow',
  'A.Napolitano',
  'R.Zimroz',
  'A.Wylomanska',
  'A.Dudek'],
 ['K.Burnecki'],
 ['P.Cizek', 'W.Härdle', 'R.Weron'],
 ['R.Weron'],
 ['P.Cizek', 'W.Härdle', 'R.Weron'],
 ['A.Izydorczyk', 'A.Janicki'],
 ['A.Weron', 'R.Weron'],
 ['A.Weron', 'R.Weron'],
 ['A.Janicki'],
 ['A.Janicki', 'A.Weron'],
 ['G.Żak'],
 ['M.Giuricich'],
 ['D.Kucharczyk'],
 ['J.Ślęzak'],
 ['M.Balcerek'],
 ['T.Zórawik'],
 ['R.Połoczański'],
 ['G.Sikora'],
 ['S.Orzeł'],
 ['J.Gajda'],
 ['Ł.Płociniczak'],
 ['M.Teuerle'],
 ['J.Janczura'],
 ['E.Broszkiewicz-Suwaj'],
 ['M.Magdziarz'],
 ['P.Miśta'],
 ['J.Iwanik'],
 ['M.Borgosz-Koczwara'],
 ['A.Wyłomańska'],
 ['P.Sztuba'],
 ['R.Weron'],
 ['J.Nowicka'],
 ['K.Burnecki'],
 ['A.Rejman'],
 ['Z.Michna'],
 ['K.Podgórski'],
 ['P.Kokoszka'],

### Prepare data

In [5]:
# Filter papers without co-authors
data = [el for el in data if len(el) > 1]

In [6]:
# Generate mapping author: #co-authored papers
authors = Counter(flatten(data))

In [7]:
# Generate edges
edges = []

for el in data:
    edges.extend(list(itertools.combinations(el, r = 2)))

In [8]:
# Node positions
G = nx.Graph()
G.add_edges_from(edges)

pos = nx.layout.spring_layout(G, scale = 2)

In [9]:
# Generate positions of nodes
node_x = []
node_y = []
node_text = []

for k, v in pos.items():
    node_x.append(v[0]); node_y.append(v[1]); node_text.append(k)

In [10]:
# Generate positions of edges
edges_x = []
edges_y = []
edges_weight = []
edges_text = []

for (u, v), weight in Counter(edges).items():
    u_x, u_y = pos[u]
    v_x, v_y = pos[v]
    edges_x.append([u_x, v_x])
    edges_y.append([u_y, v_y])
    edges_weight.append(weight)
    edges_text.append(f"{u} - {v}")

In [11]:
# Generate additional data
node_size = [authors[el]+4 for el in node_text]
customdata = [[authors[el]] for el in node_text]

### Generate plot

In [12]:
# Nodes
hovertemplate_label = "<b>%{text}</b><br>"
hovertemplate_papers = 'Number of co-authored papers: %{customdata[0]}'
hovertemplate_extra = "<extra></extra>"
hovertemplate = hovertemplate_label + hovertemplate_papers + hovertemplate_extra

nodes_trace = go.Scatter(
    x = node_x,
    y = node_y,
    text = node_text,
    hovertemplate=hovertemplate,
    customdata=customdata,
    mode = 'markers',
    marker = {
        'size': node_size
    }
)

# Edges

edges_traces = [
    go.Scatter(
        x = np.linspace(*x),
        y = np.linspace(*y),
        line = {
            'width': width,
            'color': '#888'
        },
        opacity=0.2,
        mode='lines',
        text=text,
        hovertemplate=f"<b>{text}</b><br>" + f'Number of co-authored papers: {width}' + hovertemplate_extra
    ) for x, y, width, text in zip(edges_x, edges_y, edges_weight, edges_text)
]

In [13]:
fig = go.Figure(
    data = [
        *edges_traces,
        nodes_trace
    ],
    layout = go.Layout(
        title='Hugo Steinhaus Center network of co-authored papers.',
        xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
        yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
        plot_bgcolor='rgba(0,0,0,0)',
        showlegend=False
    )
)

fig.write_html('HSC-Publications.html')